In [2]:
%pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 7.2 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade torchvision

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install numpy torch torchvision opencv-python albumentations

  Using cached torch-2.4.0-cp312-cp312-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached torchvision-0.19.0-cp312-cp312-manylinux1_x86_64.whl.metadata (6.0 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.

ERROR: Could not find a version that satisfies the requirement torch==X.Y.Z+cu121 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0)
ERROR: No matching distribution found for torch==X.Y.Z+cu121
Note: you may need to restart the kernel to use updated packages.


In [3]:
%conda install pytorch

Channels:
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/mira/anaconda3

  added / updated specs:
    - pytorch


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-24.7.1               |  py312h06a4308_0         1.2 MB
    pytorch-2.3.0              |cpu_py312h1f09096_0        76.4 MB
    ------------------------------------------------------------
                                           Total:        77.6 MB

The following NEW packages will be INSTALLED:

  pytorch            pkgs/main/linux-64::pytorch-2.3.0-cpu_py312h1f09096_0 

The following packages will be UPDATED:

  ca-certificates                      2024.3.11-h06a4308_0 --> 2024.7.2-h06a4308_0 
  certifi                          2024.6.2-py312h06a4308_0 --> 2024.7.4-py312h06a4308_0 
  conda                              24.5.0-py312h06a4308_0 --> 24.7.1-py312h0

In [2]:
import os
import json
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models.segmentation import deeplabv3_resnet50
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 데이터셋 클래스 정의
class AutonomousDrivingDataset(Dataset):
    def __init__(self, root_dir, original_transform=None, augment_transform=None):
        self.root_dir = root_dir
        self.original_transform = original_transform
        self.augment_transform = augment_transform
        self.images_dir = os.path.join(root_dir, 'images')
        self.labels_dir = os.path.join(root_dir, 'labels')
        self.image_files = sorted(os.listdir(self.images_dir))

    def __len__(self):
        return len(self.image_files) * 2  # 원본 + 증강 데이터

    def __getitem__(self, idx):
        real_idx = idx // 2
        is_augmented = idx % 2 == 1

        img_name = self.image_files[real_idx]
        img_path = os.path.join(self.images_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.jpg', '.json'))

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        with open(label_path, 'r') as f:
            label_data = json.load(f)

        annotations = sorted(label_data['Annotation'], key=lambda x: label_data['Annotation'].index(x), reverse=True)
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        class_mapping = {
            'road': 1, 'sidewalk': 2, 'road roughness': 3, 'road boundaries': 4, 'crosswalks': 5,
            'lane': 6, 'road color guide': 7, 'road marking': 8, 'parking': 9, 'traffic sign': 10,
            'traffic light': 11, 'pole/structural object': 12, 'building': 13, 'tunnel': 14,
            'bridge': 15, 'pedestrian': 16, 'vehicle': 17, 'bicycle': 18, 'motorcycle': 19,
            'personal mobility': 20, 'dynamic': 21, 'vegetation': 22, 'sky': 23, 'static': 24
        }

        for annotation in annotations:
            points = np.array(annotation['data'][0]).reshape((-1, 1, 2)).astype(np.int32)
            class_name = annotation['class_name']
            class_id = class_mapping.get(class_name, 0)
            cv2.fillPoly(mask, [points], class_id)

        if is_augmented and self.augment_transform:
            augmented = self.augment_transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        elif self.original_transform:
            augmented = self.original_transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.long()

# 데이터 증강 정의
train_transform = A.Compose([
    A.RandomRotate90(),
    A.Flip(),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.OneOf([
        A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.8, alpha_coef=0.1, p=0.5),
        A.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1, drop_color=(200, 200, 200), p=0.5),
        A.RandomShadow(num_shadows_lower=1, num_shadows_upper=3, shadow_dimension=5, shadow_roi=(0, 0.5, 1, 1), p=0.5),
    ], p=0.3),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.Resize(512, 512),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 원본 데이터를 위한 transform
original_transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 데이터셋 및 데이터로더 생성
train_dataset = AutonomousDrivingDataset(root_dir='training', original_transform=original_transform, augment_transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)

# 모델 정의 (ResNet18을 백본으로 사용)
model = deeplabv3_resnet50(weights=None, num_classes=25)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 학습 함수
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for images, masks in dataloader:
        images, masks = images.to(device), masks.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

# 검증 함수
def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, masks in dataloader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)['out']
            loss = criterion(outputs, masks)
            total_loss += loss.item()
    return total_loss / len(dataloader)

# Early stopping 클래스
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# 학습 실행
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 24
early_stopping = EarlyStopping(patience=5, min_delta=0.001)
best_val_loss = float('inf')

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss = validate(model, train_loader, criterion, device)  # 실제로는 별도의 검증 데이터셋을 사용해야 합니다
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # 최상의 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_autonomous_driving_segmentation_model.pth')
        print(f"Saved best model at epoch {epoch+1}")
    
    # Early stopping 체크
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping at epoch {epoch+1}")
        break
    
    # 20 에폭마다 모델 저장
    if (epoch + 1) % 3 == 0:
        torch.save(model.state_dict(), f'autonomous_driving_segmentation_model_epoch_{epoch+1}.pth')
        print(f"Saved model at epoch {epoch+1}")

# 최종 모델 저장
torch.save(model.state_dict(), 'final_autonomous_driving_segmentation_model.pth')
print("Training completed")

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [1]:
import os
import json
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models.segmentation import deeplabv3_resnet50
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 데이터셋 클래스 정의
class AutonomousDrivingDataset(Dataset):
    def __init__(self, root_dir, original_transform=None, augment_transform=None):
        self.root_dir = root_dir
        self.original_transform = original_transform
        self.augment_transform = augment_transform
        self.images_dir = os.path.join(root_dir, 'images')
        self.labels_dir = os.path.join(root_dir, 'labels')
        self.image_files = sorted(os.listdir(self.images_dir))

    def __len__(self):
        return len(self.image_files) * 2  # 원본 + 증강 데이터

    def __getitem__(self, idx):
        real_idx = idx // 2
        is_augmented = idx % 2 == 1

        img_name = self.image_files[real_idx]
        img_path = os.path.join(self.images_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.jpg', '.json'))

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        with open(label_path, 'r') as f:
            label_data = json.load(f)

        annotations = sorted(label_data['Annotation'], key=lambda x: label_data['Annotation'].index(x), reverse=True)
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        class_mapping = {
            'road': 1, 'sidewalk': 2, 'road roughness': 3, 'road boundaries': 4, 'crosswalks': 5,
            'lane': 6, 'road color guide': 7, 'road marking': 8, 'parking': 9, 'traffic sign': 10,
            'traffic light': 11, 'pole/structural object': 12, 'building': 13, 'tunnel': 14,
            'bridge': 15, 'pedestrian': 16, 'vehicle': 17, 'bicycle': 18, 'motorcycle': 19,
            'personal mobility': 20, 'dynamic': 21, 'vegetation': 22, 'sky': 23, 'static': 24
        }

        for annotation in annotations:
            points = np.array(annotation['data'][0]).reshape((-1, 1, 2)).astype(np.int32)
            class_name = annotation['class_name']
            class_id = class_mapping.get(class_name, 0)
            cv2.fillPoly(mask, [points], class_id)

        if is_augmented and self.augment_transform:
            augmented = self.augment_transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        elif self.original_transform:
            augmented = self.original_transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.long()

# 데이터 증강 정의
train_transform = A.Compose([
    A.RandomRotate90(),
    A.Flip(),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.OneOf([
        A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.8, alpha_coef=0.1, p=0.5),
        A.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1, drop_color=(200, 200, 200), p=0.5),
        A.RandomShadow(num_shadows_lower=1, num_shadows_upper=3, shadow_dimension=5, shadow_roi=(0, 0.5, 1, 1), p=0.5),
    ], p=0.3),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.Resize(256, 256),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 원본 데이터를 위한 transform
original_transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 데이터셋 및 데이터로더 생성
train_dataset = AutonomousDrivingDataset(root_dir='training', original_transform=original_transform, augment_transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

# 모델 정의 및 수정
model = deeplabv3_resnet50(weights=None, num_classes=25)

# 채널 수를 줄이기 위해 첫 번째 컨볼루션 레이어 수정
model.backbone.conv1 = nn.Conv2d(3, 32, kernel_size=7, stride=2, padding=3, bias=False)

# ASPP의 채널 수 감소
model.classifier[0] = nn.Conv2d(2048, 128, 1, bias=False)
model.classifier[1] = nn.BatchNorm2d(128)
model.classifier[2] = nn.ReLU()
model.classifier[3] = nn.Conv2d(128, 128, 3, padding=1, bias=False)
model.classifier[4] = nn.BatchNorm2d(128)
model.classifier[5] = nn.ReLU()
model.classifier[6] = nn.Conv2d(128, 25, 1)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for images, masks in dataloader:
        images, masks = images.to(device), masks.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

# 검증 함수
def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, masks in dataloader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)['out']
            loss = criterion(outputs, masks)
            total_loss += loss.item()
    return total_loss / len(dataloader)

# Early stopping 클래스
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# 학습 실행
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 24
early_stopping = EarlyStopping(patience=5, min_delta=0.001)
best_val_loss = float('inf')

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss = validate(model, train_loader, criterion, device)  # 실제로는 별도의 검증 데이터셋을 사용해야 합니다
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # 최상의 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_autonomous_driving_segmentation_model.pth')
        print(f"Saved best model at epoch {epoch+1}")
    
    # Early stopping 체크
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping at epoch {epoch+1}")
        break
    
    # 3 에폭마다 모델 저장
    if (epoch + 1) % 3 == 0:
        torch.save(model.state_dict(), f'autonomous_driving_segmentation_model_epoch_{epoch+1}.pth')
        print(f"Saved model at epoch {epoch+1}")

# 최종 모델 저장
torch.save(model.state_dict(), 'final_autonomous_driving_segmentation_model.pth')
print("Training completed")

IndexError: index 5 is out of range

In [4]:
%conda uninstall pytorch torchvision
%conda install pytorch torchvision


PackagesNotFoundError: The following packages are missing from the target environment:
  - torchvision



Note: you may need to restart the kernel to use updated packages.
Channels:
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/mira/anaconda3

  added / updated specs:
    - pytorch
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ffmpeg-6.1.1               |       h4c62175_0         9.6 MB
    leptonica-1.82.0           |       h42c8aad_2         2.5 MB
    libogg-1.3.5               |       h27cfd23_1         199 KB
    libtheora-1.1.1            |       h7f8727e_3         338 KB
    libvorbis-1.3.7            |       h7b6447c_0         398 KB
    libvpx-1.13.1              |       h6a678d5_0         1.0 MB
    libwebp-1.3.2              |       h11a3e52_0          87 KB
    tesseract-5.2.0            |   

In [3]:
import os
import json
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models.segmentation import deeplabv3_resnet50
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 데이터셋 클래스 정의
class AutonomousDrivingDataset(Dataset):
    def __init__(self, root_dir, original_transform=None, augment_transform=None):
        self.root_dir = root_dir
        self.original_transform = original_transform
        self.augment_transform = augment_transform
        self.images_dir = os.path.join(root_dir, 'images')
        self.labels_dir = os.path.join(self.root_dir, 'labels')
        self.image_files = sorted(os.listdir(self.images_dir))

    def __len__(self):
        return len(self.image_files) * 2  # 원본 + 증강 데이터

    def __getitem__(self, idx):
        real_idx = idx // 2
        is_augmented = idx % 2 == 1

        img_name = self.image_files[real_idx]
        img_path = os.path.join(self.images_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.jpg', '.json'))

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        with open(label_path, 'r') as f:
            label_data = json.load(f)

        annotations = sorted(label_data['Annotation'], key=lambda x: label_data['Annotation'].index(x), reverse=True)
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        class_mapping = {
            'road': 1, 'sidewalk': 2, 'road roughness': 3, 'road boundaries': 4, 'crosswalks': 5,
            'lane': 6, 'road color guide': 7, 'road marking': 8, 'parking': 9, 'traffic sign': 10,
            'traffic light': 11, 'pole/structural object': 12, 'building': 13, 'tunnel': 14,
            'bridge': 15, 'pedestrian': 16, 'vehicle': 17, 'bicycle': 18, 'motorcycle': 19,
            'personal mobility': 20, 'dynamic': 21, 'vegetation': 22, 'sky': 23, 'static': 24
        }

        for annotation in annotations:
            points = np.array(annotation['data'][0]).reshape((-1, 1, 2)).astype(np.int32)
            class_name = annotation['class_name']
            class_id = class_mapping.get(class_name, 0)
            cv2.fillPoly(mask, [points], class_id)

        if is_augmented and self.augment_transform:
            augmented = self.augment_transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        elif self.original_transform:
            augmented = self.original_transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.long()

# 데이터 증강 정의
train_transform = A.Compose([
    A.RandomRotate90(),
    A.Flip(),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.OneOf([
        A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.8, alpha_coef=0.1, p=0.5),
        A.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1, drop_color=(200, 200, 200), p=0.5),
        A.RandomShadow(num_shadows_lower=1, num_shadows_upper=3, shadow_dimension=5, shadow_roi=(0, 0.5, 1, 1), p=0.5),
    ], p=0.3),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.Resize(256, 256),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 원본 데이터를 위한 transform
original_transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 데이터셋 및 데이터로더 생성
train_dataset = AutonomousDrivingDataset(root_dir='training', original_transform=original_transform, augment_transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

# 모델 정의 및 수정
model = deeplabv3_resnet50(weights=None, num_classes=25)

# 첫 번째 컨볼루션 레이어 수정
model.backbone.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)

# classifier를 새롭게 정의하여 오류 방지
model.classifier = nn.Sequential(
    nn.Conv2d(2048, 256, 1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.Conv2d(256, 256, 3, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.Conv2d(256, 25, 1)
)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for images, masks in dataloader:
        images, masks = images.to(device), masks.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

# 검증 함수
def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, masks in dataloader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)['out']
            loss = criterion(outputs, masks)
            total_loss += loss.item()
    return total_loss / len(dataloader)

# Early stopping 클래스
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# 학습 실행
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 24
early_stopping = EarlyStopping(patience=5, min_delta=0.001)
best_val_loss = float('inf')

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss = validate(model, train_loader, criterion, device)  # 실제로는 별도의 검증 데이터셋을 사용해야 합니다
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # 최상의 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_autonomous_driving_segmentation_model.pth')
        print(f"Saved best model at epoch {epoch+1}")
    
    # Early stopping 체크
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping at epoch {epoch+1}")
        break
    
    # 3 에폭마다 모델 저장
    if (epoch + 1) % 3 == 0:
        torch.save(model.state_dict(), f'autonomous_driving_segmentation_model_epoch_{epoch+1}.pth')
        print(f"Saved model at epoch {epoch+1}")

# 최종 모델 저장
torch.save(model.state_dict(), 'final_autonomous_driving_segmentation_model.pth')
print("Training complete. Final model saved.")



Premature end of JPEG file
Premature end of JPEG file


In [ ]:
import os
import json
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 데이터셋 클래스 정의
class AutonomousDrivingDataset(Dataset):
    def __init__(self, root_dir, original_transform=None, augment_transform=None):
        self.root_dir = root_dir
        self.original_transform = original_transform
        self.augment_transform = augment_transform
        self.images_dir = os.path.join(root_dir, 'images')
        self.labels_dir = os.path.join(self.root_dir, 'labels')
        self.image_files = sorted(os.listdir(self.images_dir))

    def __len__(self):
        return len(self.image_files) * 2  # 원본 + 증강 데이터

    def __getitem__(self, idx):
        real_idx = idx // 2
        is_augmented = idx % 2 == 1

        img_name = self.image_files[real_idx]
        img_path = os.path.join(self.images_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.jpg', '.json'))

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        with open(label_path, 'r') as f:
            label_data = json.load(f)

        annotations = sorted(label_data['Annotation'], key=lambda x: label_data['Annotation'].index(x), reverse=True)
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        class_mapping = {
            'road': 1, 'sidewalk': 2, 'road roughness': 3, 'road boundaries': 4, 'crosswalks': 5,
            'lane': 6, 'road color guide': 7, 'road marking': 8, 'parking': 9, 'traffic sign': 10,
            'traffic light': 11, 'pole/structural object': 12, 'building': 13, 'tunnel': 14,
            'bridge': 15, 'pedestrian': 16, 'vehicle': 17, 'bicycle': 18, 'motorcycle': 19,
            'personal mobility': 20, 'dynamic': 21, 'vegetation': 22, 'sky': 23, 'static': 24
        }

        for annotation in annotations:
            points = np.array(annotation['data'][0]).reshape((-1, 1, 2)).astype(np.int32)
            class_name = annotation['class_name']
            class_id = class_mapping.get(class_name, 0)
            cv2.fillPoly(mask, [points], class_id)

        if is_augmented and self.augment_transform:
            augmented = self.augment_transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        elif self.original_transform:
            augmented = self.original_transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.long()

# 데이터 증강 정의
train_transform = A.Compose([
    A.RandomRotate90(),
    A.Flip(),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.OneOf([
        A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.8, alpha_coef=0.1, p=0.5),
        A.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1, drop_color=(200, 200, 200), p=0.5),
        A.RandomShadow(num_shadows_lower=1, num_shadows_upper=3, shadow_dimension=5, shadow_roi=(0, 0.5, 1, 1), p=0.5),
    ], p=0.3),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.Resize(256, 256),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 원본 데이터를 위한 transform
original_transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 데이터셋 및 데이터로더 생성
train_dataset = AutonomousDrivingDataset(root_dir='training', original_transform=original_transform, augment_transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

# 모델 정의 및 수정
model = deeplabv3_mobilenet_v3_large(pretrained=False, num_classes=25)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for images, masks in dataloader:
        images, masks = images.to(device), masks.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

# 검증 함수
def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, masks in dataloader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)['out']
            loss = criterion(outputs, masks)
            total_loss += loss.item()
    return total_loss / len(dataloader)

# Early stopping 클래스
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# 학습 실행
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 13
early_stopping = EarlyStopping(patience=5, min_delta=0.001)
best_val_loss = float('inf')

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss = validate(model, train_loader, criterion, device)  # 실제로는 별도의 검증 데이터셋을 사용해야 합니다
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # 최상의 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_autonomous_driving_segmentation_model.pth')
        print(f"Saved best model at epoch {epoch+1}")
    
    # Early stopping 체크
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping at epoch {epoch+1}")
        break
    
    # 3 에폭마다 모델 저장
    if (epoch + 1) % 3 == 0:
        torch.save(model.state_dict(), f'autonomous_driving_segmentation_model_epoch_{epoch+1}.pth')
        print(f"Saved model at epoch {epoch+1}")

# 최종 모델 저장
torch.save(model.state_dict(), 'final_autonomous_driving_segmentation_model.pth')
print("Training complete. Final model saved.")
